In [18]:
import faiss
import os
import pandas as pd
import numpy as np
import requests
import pyphi

In [11]:
def call_embedding_ws(names):
#    url = 'http://54.36.53.127:8009/embedding'
    url = 'http://127.0.0.1:8009/embedding'
    headers = {"content-type": "application/json"}
    response = requests.post(url, json=names)
    arr = pyphi.jsonify.loads(response.text)
    x = np.array(arr)
    x = x.astype(dtype=np.float32)
    return x

def reverse_name(name):
    a=name.split()
    a.reverse()
    return " ".join(a)

def cos_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    return dot_product

def sort_by_distance(query_vector, result_vectors):
    for v in result_vectors:
        cos_similarity(query_vector, v)
        
def is_arabic(name):
    res = re.findall(r'[\u0600-\u06FF]+',name)
    if len(res) == 0:
        return False
    else:
        return True

In [24]:
source_vectors = pd.read_csv('/data/test-sets/eng_eng/vectors_pos_eng_eng_2x3.tsv', sep='\t', usecols=[2,3,4,5])

In [44]:
left_vector_strings = source_vectors['left_vector'].as_matrix()
left_vectors = np.zeros(shape=(len(left_vector_strings),256))
i = 0
for v in left_vector_strings:
    x = np.fromstring(v, dtype=np.float32, sep=',')
    left_vectors[i] = x
    i = i+1
left_vectors = left_vectors.astype('float32')
left_vector_strings = None

In [45]:
len(source_vectors)

11144

In [46]:
d = 256
index = faiss.IndexFlatL2(d)

In [47]:
index

<swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f502dd8ffc0> >

In [48]:
print(index.is_trained)
index.add(left_vectors)                  # add vectors to the index
print(index.ntotal)

True
11144


In [51]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

k = 5                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print("I: {i}".format(i=I))
print("D: {d}".format(d=D))
D, I = index.search(xq, k)     # actual search
print("I: {i}".format(i=I[:5]))                    # neighbors of the 5 first queries
print("I: {i}".format(i=I[-5:]))                   # neighbors of the 5 last queries

True
100000
I: [[  0 393 363  78 924]
 [  1 555 277 364 617]
 [  2 304 101  13 801]
 [  3 173  18 182 484]
 [  4 288 370 531 178]]
D: [[ 0.          7.17517328  7.2076292   7.25116253  7.32189512]
 [ 0.          6.32356453  6.6845808   6.79994535  6.88447952]
 [ 0.          5.79640865  6.39173603  7.28151226  7.64050198]
 [ 0.          7.27790546  7.52798653  7.66284657  7.78595734]
 [ 0.          6.76380348  7.29512024  7.36881447  7.3900466 ]]
I: [[ 381  207  210  477  588]
 [ 526  911  142   72  300]
 [ 838  527 1290  425  281]
 [ 196  184  164  359  466]
 [ 526  377  120  425  545]]
I: [[ 9900 10500  9309  9831  9810]
 [11055 10895 10812 11321 10260]
 [11353 11103 10164  9787 10719]
 [10571 10664 10632  9638  9589]
 [ 9628  9554 10036  9582 10304]]
